In [ ]:
import hyperspy.api as hs
import os
import cv2
import numpy as np
import scipy as sc
import scipy.io as sio
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm

In [ ]:
%pylab qt

# Saving Signal 

In [2]:
pathFile = "D:/Users/Don/IntensityImages__C16_15_24_151205_030__2015_12_12_01/"

In [3]:
gainMapFile = "D:/Users/Don/Gain Map/GainMap_FZJ_80keV_binning2_gain2.mat"

In [4]:
newFileName = "Any_name" #Name of hdf5 file 

In [ ]:
save_signal(pathFile, gainMapFile, newFileName)

# Performing Decomposition

In [ ]:
sample = newFileName + ".hdf5"

In [ ]:
run_decomposition(sample)

# 1st Operation: Systematic error shift contribution cancellation

In [ ]:
sample_1 = hs.load("Sample_1.hdf5")

In [ ]:
sample_2 = hs.load("Sample_2.hdf5")

In [ ]:
file_nbr = 1

In [ ]:
file_nbr_2 = 2

## Align relative to ref data

In [ ]:
Translation_mat = align2D_cv_2stacks(sample_1, sample_2, cv2.MOTION_TRANSLATION)

In [ ]:
np.save('Lorentz_STEM_'+str(file_nbr)+'_ref_shift.npy', Translation_mat)

In [ ]:
#For Positive Bias
plot_save('Lorentz_STEM_'+str(file_nbr), 'ref_shift', sample_1.axes_manager[1].size, sample_1.axes_manager[0].size)

In [ ]:
#For Negative Bias
plot_save_2('Lorentz_STEM_'+str(file_nbr), 'ref_shift', sample_2.axes_manager[1].size, sample_2.axes_manager[0].size)

## 2nd Operation: Plotting field map with mean inner potential shift contribution cancelled


In [ ]:
#For Positive Bias
plot_save_3('Lorentz_STEM_'+str(file_nbr), 'Lorentz_STEM_'+str(file_nbr_2), \
            'ref_shift', sample_1.axes_manager[1].size, sample_1.axes_manager[0].size)

In [ ]:
#For Negative Bias
plot_save_4('Lorentz_STEM_'+str(file_nbr), 'Lorentz_STEM_'+str(file_nbr_2), \
            'ref_shift', sample_2.axes_manager[1].size, sample_2.axes_manager[0].size)

In [ ]:
#To view the figures and signals of the solar cell dataset 

def save_signal(pathFile, gainMapFile, newFileName):
    dataDir = pathfile
    mats = []
    for file in os.listdir(dataDir):
        mats.append(sio.loadmat(dataDir+file))
        
    for n in range (lens(mats)):
        mats[n] = np.swapaxes(mats[n]['a3Image16'],2,0)
        
    Temp = np.memmap('Temporary_memory.mymemmap', dtype = 'float32', mode = 'w+', \
                     shape = (256,256,264,132))

    Temp_2 = np.memmap('Temporary_memory_2.mymemmap', dtype = 'float32', mode = 'w+', \
                       shape = (65536,264,132))
    
    arr = (mats[0], mats[1], mats[2], mats[3],mats[4],mats[5],mats[6],mats[7],mats[8],mats[9],\
           mats[10],mats[11],mats[12],mats[13])
    Temp_2 = np.vstack(arr)
    
    if gainMap:
        gain_map = gainMapFile
        gain_map_1 = sio.loadmat(gain_map)
        gain_map_1 = gain_map_1['GainMap']
        gain_map_1 = np.swapaxes(gain_map_1,0,1)
        
        for n in range (65536):
            Temp_2[n] = np.multiply(Temp_2[n], gain_map_1)
        
        for i in range(int((np.size(Temp_2,0)/256)+1)):
            for j in range(256):
                if (i*256 + j) >= np.size(Temp_2,0):
                    End_i = i
                    End_j = j
                    break
                else:
                    Temp[i][j][:] = Temp_2[i*256+j][:]
                
    
    else:
        for i in range(int((np.size(Temp_2,0)/256)+1)):
            for j in range(256):
                if (i*256 + j) >= np.size(Temp_2,0):
                    End_i = i
                    End_j = j
                    break
                else:
                    Temp[i][j][:] = Temp_2[i*256+j][:]
                    
    #Plotting the signal
    
    Signal2D = hs.signals.Signal2D(Temp).as_lazy()
    Signal2D.plot()
    print ("Save the signal plot? Y/N ")
    x = input()
    if x == "Y":
        Signal2D.save(newFileName + 'hdf5')
    else:
        pass
    
    
    del Temp
    del Temp_2

In [ ]:
#To perform decomposition on the dataset

def run_decomposition(sample):
    sample = hs.load(sample)
    sample.plot()
    
    print("Do you want to plot the region of interest? Y/N ")
    Answer = input()
    if Answer == "Y":
        print("Enter the coordinate for a, b, c ,d:")
        a = int(input("a: "))
        b = int(input("b: "))
        c = int(input("c: "))
        d = int(input("d: "))
        ROI = hs.roi.RectangularROI(left = a, right = b, top = c, bottom = d)
        IMC = ROI.interactive(sample)
        IMC.plot()
        IMC.decompostion()
    else:
        sample.decomposition()
        
    print ("Do you want to plot the \"Scree Plot\"? Y/N ")
    Answer_2 = input ()
    Answer_3 = int(input("How many components you wish to be mentioned in the scree plot? "))
    Answer_4 = int(input("How many components do you wish to plot? "))
    if Answer_2 == "Y":
        if sample:
            ax = sample.plot_explained_variance_ratio(xaxis_type = 'number', n = Answer_3)
            sc = sample.get_decomposition_model(Answer_4)
            sc.plot()
        elif IMC:
            ax = IMC.plot_explained_variance_ratio(xaxis_type= 'number', n = Answer_3)
            sc = IMC.get_decomposition_model(Answer_4)
            sc.plot()
        else:
            pass

In [ ]:
def align2D_cv_2stacks(hs_stack1, hs_stack2, warp_mode):
    
    hs_stack1.change_dtype('float32')
    hs_stack2.change_dtype('float32')
    
    hs_stack1.unfold_navigation_space()
    hs_stack2.unfold_navigation_space()
         
    # High pass filter using low pass and Gaussian blurring
    filter_1 = ndimage.gaussian_filter(hs_stack1.data[0], 10)
    # Convert Numpy hpf data in openCV format (3 channels)
    img_1_hpf_cv = cv2.cvtColor(filter_1, cv2.COLOR_GRAY2BGR)
    # Convert openCV format (3 channels) in openCV format grey (1 channel)
    img_1_hpf_cv = cv2.cvtColor(img_1_hpf_cv, cv2.COLOR_BGR2GRAY)
     
    #find the shape
    sz = img_1_hpf_cv.shape
    
    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else :
        warp_matrix = np.eye(2, 3, dtype=np.float32)
        
    warp_matrix_returned = np.empty((hs_stack1.data.shape[0], warp_matrix.shape[0], warp_matrix.shape[1]))
    
    # Specify the number of iterations.
    number_of_iterations = 3000;
    
    # Specify the threshold of the increment
    termination_eps = 1e-10;
    
    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)

    for i in range (0,hs_stack1.data.shape[0]):
        # High pass filter using low pass and Gaussian blurring 
        filter_1 = ndimage.gaussian_filter(hs_stack1.data[i], 10)
        # Convert Numpy hpf data in openCV format (3 channels)
        img_1_hpf_cv = cv2.cvtColor(filter_1, cv2.COLOR_GRAY2BGR)
        # Convert openCV format (3 channels) in openCV format grey (1 channel)
        img_1_hpf_cv = cv2.cvtColor(img_1_hpf_cv, cv2.COLOR_BGR2GRAY)
        
        ### Idem for ref data
        # High pass filter using low pass and Gaussian blurring         
        filter_2 = ndimage.gaussian_filter(hs_stack2.data[i], 10)
        # Convert Numpy hpf data in openCV format (3 channels)
        img_2_hpf_cv = cv2.cvtColor(filter_2, cv2.COLOR_GRAY2BGR)
        # Convert openCV format (3 channels) in openCV format grey (1 channel)
        img_2_hpf_cv = cv2.cvtColor(img_2_hpf_cv, cv2.COLOR_BGR2GRAY)
    
        # Run the ECC algorithm. The results are stored in warp_matrix.
        (cc, warp_matrix) = cv2.findTransformECC(img_1_hpf_cv,img_2_hpf_cv,warp_matrix, warp_mode, criteria, None, 1)
    
        warp_matrix_returned[i,:,:] = warp_matrix
        
        print(str(i) +' / '+str(hs_stack1.data.shape[0]-1))
              
    hs_stack1.fold()
    hs_stack2.fold()
    
    return warp_matrix_returned

In [ ]:
#Plotting Field Map for 1st Operation: Systematic error contribution cancellation (for positive bias)

def plot_save(path_analysis, typeal, sizex, sizey):

    if  typeal == 'relative_shift' or \
        typeal == 'ref_relative_shift' or \
        typeal == 'ref_shift':
            Translation_mat = np.load(path_analysis + '_' + typeal + '.npy')
            Translation_mat_square = np.reshape(Translation_mat, (sizex,sizey,2,3))
            Shift_mat = Translation_mat_square[:,:,:,2]
    else:
        print('pb')
            
    M = np.empty_like(Shift_mat[:,:,0])
    for i in range(0,np.size(Shift_mat[:,0,0])):
        for j in range(0,np.size(Shift_mat[i,:,0])):
            M[i,j] =np.sqrt(np.power(Shift_mat[i,j,0],2)+np.power(Shift_mat[i,j,1],2))
        
    Mi_max, Mj_max = np.unravel_index(np.argmax(M, axis=None), M.shape)

    X = np.arange(-np.size(Shift_mat[0,:,0])/2, np.size(Shift_mat[0,:,0])/2, 1)
    Y = np.arange(-np.size(Shift_mat[:,0,0])/2, np.size(Shift_mat[:,0,0])/2, 1)

    fig, ax = plt.subplots(squeeze=True, figsize=(8, 8))

    # Contour Plot
    cp = plt.contourf(X, -Y, M, cmap=cm.coolwarm)
    cb = plt.colorbar(cp)

    q = ax.quiver(X, -Y, Shift_mat[:,:,1]/M[Mi_max,Mj_max], Shift_mat[:,:,0]/M[Mi_max,Mj_max], \
        units='x', width=0.15, scale=0.3)
    fig.savefig(path_analysis + '_' + typeal +'.png', dpi=300)

In [ ]:
#Plotting Field Map for 1st Operation: Systematic error contribution cancellation (for negative bias)

def plot_save_2(path_analysis, typeal, sizex, sizey):

    if  typeal == 'relative_shift' or \
        typeal == 'ref_relative_shift' or \
        typeal == 'ref_shift':
            Translation_mat = np.load(path_analysis + '_' + typeal + '.npy')
            Translation_mat_square = np.reshape(Translation_mat, (sizex,sizey,2,3))
            Shift_mat = Translation_mat_square[:,:,:,2]
    else:
        print('pb')
            
    M = np.empty_like(Shift_mat[:,:,0])
    for i in range(0,np.size(Shift_mat[:,0,0])):
        for j in range(0,np.size(Shift_mat[i,:,0])):
            M[i,j] =np.sqrt(np.power(Shift_mat[i,j,0],2)+np.power(Shift_mat[i,j,1],2))
                    
    Mi_max, Mj_max = np.unravel_index(np.argmax(M, axis=None), M.shape)

    X = np.arange(-np.size(Shift_mat[0,:,0])/2, np.size(Shift_mat[0,:,0])/2, 1)
    Y = np.arange(-np.size(Shift_mat[:,0,0])/2, np.size(Shift_mat[:,0,0])/2, 1)

    fig, ax = plt.subplots(squeeze=True, figsize=(8, 8))

    # Contour Plot
    cp = plt.contourf(X, -Y, M, cmap=cm.coolwarm)
    cb = plt.colorbar(cp)

    q = ax.quiver(X, -Y, -Shift_mat[:,:,1]/M[Mi_max,Mj_max], -Shift_mat[:,:,0]/M[Mi_max,Mj_max], \
        units='x', width=0.15, scale=0.3)
    fig.savefig(path_analysis + '_' + typeal +'.png', dpi=300)

In [ ]:
# Plotting Field Map for 2nd Operation: Mean inner potential shift contribution cancellation (for positive bias)

def plot_save_3(path_analysis, path_analysis2, typeal, sizex, sizey):
    
    if  typeal == 'relative_shift' or \
        typeal == 'ref_relative_shift' or \
        typeal == 'ref_shift':
            Translation_mat = np.load(path_analysis + '_' + typeal + '.npy')
            Translation_mat_square = np.reshape(Translation_mat, (sizex,sizey,2,3))
            Shift_mat = Translation_mat_square[:,:,:,2]
            
            Translation_mat_2 = np.load(path_analysis2 + '_' + typeal + '.npy')
            Translation_mat_square_2 = np.reshape(Translation_mat, (sizex,sizey,2,3))
            Shift_mat_2 = Translation_mat_square[:,:,:,2]
    else:
        print('pb')
    
    N = np.empty_like(Shift_mat[:,:,0])
    L = np.empty_like(Shift_mat[:,:,0])
    M = np.empty_like(Shift_mat[:,:,0])
    for i in range(0,np.size(Shift_mat[:,0,0])):
        for j in range(0,np.size(Shift_mat[i,:,0])):
            N[i,j] = np.sqrt(np.power(Shift_mat[i,j,0],2)+np.power(Shift_mat[i,j,1],2))
            L[i,j] = np.sqrt(np.power(Shift_mat_2[i,j,0],2)+np.power(Shift_mat_2[i,j,1],2))
            M[i,j] = N[i,j] - L[i,j]
        
    Mi_max, Mj_max = np.unravel_index(np.argmax(M, axis=None), M.shape)

    X = np.arange(-np.size(Shift_mat[0,:,0])/2, np.size(Shift_mat[0,:,0])/2, 1)
    Y = np.arange(-np.size(Shift_mat[:,0,0])/2, np.size(Shift_mat[:,0,0])/2, 1)

    fig, ax = plt.subplots(squeeze=True, figsize=(8, 8))

    # Contour Plot
    cp = plt.contourf(X, -Y, M, cmap=cm.coolwarm)
    cb = plt.colorbar(cp)

    q = ax.quiver(X, -Y, Shift_mat[:,:,1]/M[Mi_max,Mj_max], Shift_mat[:,:,0]/M[Mi_max,Mj_max], \
        units='x', width=0.15, scale=0.3)
    fig.savefig(path_analysis + '_' + typeal + '2.png', dpi=300)


In [ ]:
# Plotting Field Map for 2nd Operation: Mean inner potential shift contribution cancellation (for negative bias)

def plot_save_4(path_analysis, path_analysis2, typeal, sizex, sizey):
    
    if  typeal == 'relative_shift' or \
        typeal == 'ref_relative_shift' or \
        typeal == 'ref_shift':
            Translation_mat = np.load(path_analysis + '_' + typeal + '.npy')
            Translation_mat_square = np.reshape(Translation_mat, (sizex,sizey,2,3))
            Shift_mat = Translation_mat_square[:,:,:,2]
            
            Translation_mat_2 = np.load(path_analysis2 + '_' + typeal + '.npy')
            Translation_mat_square_2 = np.reshape(Translation_mat, (sizex,sizey,2,3))
            Shift_mat_2 = Translation_mat_square[:,:,:,2]
    else:
        print('pb')
    
    N = np.empty_like(Shift_mat[:,:,0])
    L = np.empty_like(Shift_mat[:,:,0])
    M = np.empty_like(Shift_mat[:,:,0])
    for i in range(0,np.size(Shift_mat[:,0,0])):
        for j in range(0,np.size(Shift_mat[i,:,0])):
            N[i,j] = np.sqrt(np.power(Shift_mat[i,j,0],2)+np.power(Shift_mat[i,j,1],2))
            L[i,j] = np.sqrt(np.power(Shift_mat_2[i,j,0],2)+np.power(Shift_mat_2[i,j,1],2))
            M[i,j] = N[i,j] - L[i,j]
        
    Mi_max, Mj_max = np.unravel_index(np.argmax(M, axis=None), M.shape)

    X = np.arange(-np.size(Shift_mat[0,:,0])/2, np.size(Shift_mat[0,:,0])/2, 1)
    Y = np.arange(-np.size(Shift_mat[:,0,0])/2, np.size(Shift_mat[:,0,0])/2, 1)

    fig, ax = plt.subplots(squeeze=True, figsize=(8, 8))

    # Contour Plot
    cp = plt.contourf(X, -Y, M, cmap=cm.coolwarm)
    cb = plt.colorbar(cp)

    q = ax.quiver(X, -Y, -Shift_mat[:,:,1]/M[Mi_max,Mj_max], -Shift_mat[:,:,0]/M[Mi_max,Mj_max], \
        units='x', width=0.15, scale=0.3)
    fig.savefig(path_analysis + '_' + typeal + '2.png', dpi=300)
